# Hail Tutorial GWAS

### 1. Setting up work environment
The first step of performing a GWAS is to load in our depedencies and set up our work environment.

In [1]:
"""
Import statements allow us to reuse code written previously by ourselves or others. 
Here we are importing the "Hail" library which is the core strategy we are going to be using to organize our data and to eventually perform statistical analyses.
"""
import hail as hl
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()
hl.init(quiet=True)

Loading BokehJS ...

2023-01-18 18:44:16 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### 2. Loading in the data
After we finish loading our dependencies, we can go ahead and start loading the data, starting with our genotype data (stored in a folder called "1kg.mt") and our phenotype data (stored in a file called "1kg_annotations.txt").

In [3]:
# Loading in the genotype data from our "data" folder and storing it in a variable called "mt", short for "MatrixTable" (one of the key innovations of the Hail library)
mt = hl.read_matrix_table('data/1kg.mt')

# Loading in the phenotype data from our "data" folder and storing it in a variable called "table"
table = hl.import_table('data/1kg_annotations.txt', impute=True).key_by('Sample')

Now that our data is loaded in, we can combine the two to form a consolidated dataset containing all the relevant information we are going to use for our analyses.

In [7]:
# We can use the "annotate_cols" function to add our phenotype data in the "table" variable 
mt = mt.annotate_cols(pheno = table[mt.s])

It is always a good idea to take a look at our data to see what format we are working with and the available information we have. One way to do this is by using the "describe" method. An example is shown below:

In [8]:
# Describing the format of the "mt variable" using an interactive widget
mt.describe(widget = True)

After running the cell above, we can now interact with the four main components of our dataset (globals, rows, cols, and entries). In Hail, each row consist of one specific genetic variant and each column consist of one specific individual. An entry is an intersection of a row and a column and contains information about a specific variant for a particular individual (such as the genetic call).

Additionally from the interactive cell above, we can see that in the "col" tab, we have two fields that we can access: "s" and "pheno". If we expand the "pheno" field, we can see what information we have available for each individual. In this dataset, we have access to the following variables for each individual: "Population", "SuperPopulation", "isFemale", "PurpleHair", and "CaffeineConsumption". 

Feel free to explore the "row" and "entry" tabs to learn more about those parts of our dataset.

### 3. Quality Control

After we load and explore our dataset, the next step is to perform some quality control (QC) so that we have a clean dataset prior to statistical analysis. In a GWAS, there are quite a few quality control measures we have to do. In order to organize ourselves, let's split these quality control measures into two categories: QC on the genotype data and QC on the phenotype data.

Let's begin with QC on the phenotype data:
1. Remove individuals with high levels of missingness (people who we do not have enough data for) (sample_qc.call_rate > 0.98)
2. Remove/modify individuals who have sex discrepencies

Let's continue on to QC on the genotype data:
1. Missingness of SNPs (remove variants for which we do not have adequate data for)
3. Minor allele frequency (MAF) (remove variants that are too rare)
4. Hardy–Weinberg equilibrium (HWE) (remove variants that deviate from Hardy-Weinberg equilibrium)


7. Population stratification

5. Heterozygosity
6. Relatedness
8. Ancestry

In [9]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display
%matplotlib inline

mt.describe(widget=True)

In [10]:
mt = hl.sample_qc(mt)
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.98))
#ab = mt.AD[1] / hl.sum(mt.AD)
#filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        #(mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        #(mt.GT.is_hom_var() & (ab >= 0.9)))
#mt = mt.filter_entries(filter_condition_ab)
mt = hl.variant_qc(mt)
original = mt
# Minor allele frequency cutoff
# mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.05)
# Hardy-Weinberg equilibrium (HWE) cutoff
# mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-6)

@widgets.interact(MAF=widgets.FloatSlider(min=0.01, max=0.05, step=0.01, value=0.05, layout = widgets.Layout(width='500px')), HWE=widgets.FloatLogSlider(value=6, base=10, min=-10, max=-6, step=1, readout_format='.2e', layout = widgets.Layout(width='500px')))
def variant_qc_interactive(MAF = 0.05, HWE=1e-6):
    global mt
    global original
    mt = original
    mt = mt.filter_rows(mt.variant_qc.AF[1] > MAF)
    mt = mt.filter_rows(mt.variant_qc.p_value_hwe > HWE)
    print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))

interactive(children=(FloatSlider(value=0.05, description='MAF', layout=Layout(width='500px'), max=0.05, min=0…

Let's quickly perform a GWAS and visualize the results!

In [5]:
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))

Samples: 224  Variants: 8003


In [6]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale])
p = hl.plot.qq(gwas.p_value)
show(p)
p = hl.plot.manhattan(gwas.p_value)
show(p)

Let's add in some PCA and control for variation in our regression.

In [7]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)
mt = mt.annotate_cols(scores = pcs[mt.s].scores)
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]])
p = hl.plot.qq(gwas.p_value)
show(p)
p = hl.plot.manhattan(gwas.p_value)
show(p)

In [8]:
gwas.show(width=100)

,,,,,,,,
locus,alleles,n,sum_x,y_transpose_x,beta,standard_error,t_stat,p_value
locus<GRCh37>,array<str>,int32,float64,float64,float64,float64,float64,float64
1:904165,"[""G"",""A""]",224,5.30e+01,2.29e+02,1.30e-02,2.00e-01,6.48e-02,9.48e-01
1:1707740,"[""T"",""G""]",224,7.40e+01,3.34e+02,2.02e-01,1.83e-01,1.11e+00,2.70e-01
1:2284195,"[""T"",""C""]",224,1.39e+02,6.36e+02,-1.07e-01,1.51e-01,-7.07e-01,4.80e-01
1:2779043,"[""T"",""C""]",224,3.32e+02,1.47e+03,2.82e-01,1.58e-01,1.79e+00,7.54e-02
1:2944527,"[""G"",""A""]",224,1.00e+02,4.47e+02,-2.78e-01,1.81e-01,-1.53e+00,1.27e-01
1:3803755,"[""T"",""C""]",224,3.23e+02,1.42e+03,-2.11e-02,1.43e-01,-1.48e-01,8.82e-01
1:4121584,"[""A"",""G""]",224,1.40e+02,6.51e+02,2.29e-02,1.36e-01,1.68e-01,8.67e-01
1:4170048,"[""C"",""T""]",224,1.11e+02,5.02e+02,4.01e-01,1.61e-01,2.50e+00,1.32e-02


In [9]:
gwas.order_by("p_value").to_pandas().head(10)

,locus,alleles,n,sum_x,y_transpose_x,beta,standard_error,t_stat,p_value
0,8:19600329,"[A, G]",224,240.000000,1149.000000,0.710940,0.129726,5.480321,1.167890e-07
1,8:19619751,"[G, A]",224,102.000000,509.000000,0.837023,0.157132,5.326892,2.483578e-07
2,8:19826373,"[G, A]",224,239.000000,1076.000000,0.657047,0.137838,4.766797,3.419653e-06
3,8:19651161,"[T, C]",224,189.000000,906.000000,0.558440,0.125995,4.432244,1.477245e-05
4,8:19943027,"[G, A]",224,97.000000,462.000000,0.780023,0.194232,4.015934,8.148142e-05
5,12:4702230,"[G, A]",224,23.103139,103.721973,1.210201,0.331844,3.646897,3.320220e-04
6,17:71113564,"[G, A]",224,41.000000,151.000000,-0.840535,0.230491,-3.646710,3.322499e-04
7,4:108530885,"[C, T]",224,165.739910,778.699552,0.525955,0.147482,3.566238,4.450898e-04
8,7:33081018,"[T, C]",224,310.385650,1405.928251,0.477917,0.136499,3.501242,5.615725e-04
9,14:49135134,"[G, A]",224,49.000000,158.000000,-0.774710,0.224183,-3.455707,6.595956e-04
